# Vocabulaire et correspondance (Matching)

Jusqu'à présent, nous avons vu comment un corps de texte est divisé en tokens, et comment les tokens individuels sont analysés et étiquetés avec des parties du discours, des dépendances et des lemmes.

Dans cette section, nous allons identifier et étiqueter des phrases spécifiques qui correspondent à des modèles que nous pouvons définir nous-mêmes.

## Correspondance basée sur des règles
spaCy propose un outil de recherche de règles appelé `Matcher` qui vous permet de construire une bibliothèque de modèles de tokens, puis de comparer ces modèles à un objet Doc pour renvoyer une liste des correspondances trouvées. Vous pouvez faire correspondre n'importe quelle partie du token, y compris le texte et les annotations, et vous pouvez ajouter plusieurs motifs au même outil de correspondance.

In [18]:
# Perform standard imports
import spacy
nlp = spacy.load('en_core_web_sm')

In [19]:
# Import the Matcher library
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

<font color=green>Ici, `matcher` est un objet qui s'apparie à l'objet `Vocab` courant. Nous pouvons ajouter et supprimer des objets de correspondance nommés spécifiques à `matcher` si nécessaire.</font>

### Créer des patterns
Dans la littérature, l'expression "solar power" peut apparaître sous la forme d'un seul mot ou de deux, avec ou sans trait d'union. Dans cette section, nous développerons un outil de recherche nommé "SolarPower" qui trouvera les trois :

In [20]:
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'LOWER': 'power'}]
pattern3 = [{'LOWER': 'solar'}, {'IS_PUNCT': True}, {'LOWER': 'power'}]

matcher.add('SolarPower', [pattern1, pattern2, pattern3])

Décomposons cela :
* `pattern1` cherche un seul token dont le texte en minuscules se lit 'solarpower'
* `pattern2` cherche deux tokens adjacents qui se lisent 'solar' et 'power' dans cet ordre.
* `pattern3` cherche trois tokens adjacents, avec un jeton central qui peut être n'importe quelle ponctuation.<font color=green>*</font>

<font color=green>\* N'oubliez pas que les espaces simples ne sont pas symbolisés et qu'ils ne sont donc pas considérés comme des signes de ponctuation.</font>
<br>Une fois que nous avons défini nos patterns, nous les passons dans `matcher` avec le nom 'SolarPower', et nous mettons *callbacks* à `None` (nous reviendrons sur les callbacks plus tard).

### Appliquer matcher à un objet Doc

In [21]:
doc = nlp(u'The Solar Power industry continues to grow as demand \
for solarpower increases. Solar-power cars are gaining popularity.')

In [22]:
found_matches = matcher(doc)
print(found_matches)

[(8656102463236116519, 1, 3), (8656102463236116519, 10, 11), (8656102463236116519, 13, 16)]


`matcher` retourne une liste de tuples. Chaque tuple contient un ID pour la correspondance, avec des tokens de début et de fin qui correspondent au span `doc[start:end]`

In [23]:
for match_id, start, end in found_matches:
    string_id = nlp.vocab.strings[match_id]  # get string representation
    span = doc[start:end]                    # get the matched span
    print(match_id, string_id, start, end, span.text)

8656102463236116519 SolarPower 1 3 Solar Power
8656102463236116519 SolarPower 10 11 solarpower
8656102463236116519 SolarPower 13 16 Solar-power


Le `match_id` est simplement la valeur de hachage de la `string_ID` 'SolarPower'

### Définition des options et des quantificateurs de motifs
Vous pouvez rendre les règles optionnelles en passant l'argument `'OP':'*'`. Cela nous permet de rationaliser notre liste de motifs :

In [24]:
# Redefine the patterns:
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP':'*'}, {'LOWER': 'power'}]

# Remove the old patterns to avoid duplication:
matcher.remove('SolarPower')

# Add the new set of patterns to the 'SolarPower' matcher:
matcher.add('SolarPower', [pattern1, pattern2])

In [25]:
found_matches = matcher(doc)
print(found_matches)

[(8656102463236116519, 1, 3), (8656102463236116519, 10, 11), (8656102463236116519, 13, 16)]


Il a trouvé les deux modèles de deux mots, avec et sans le trait d'union !

Les quantificateurs suivants peuvent être transmis à la fonction `'OP'` key:
<table><tr><th>OP</th><th>Description</th></tr>

<tr ><td><span >\!</span></td><td>Négation du motif, en exigeant qu'il corresponde exactement à 0 fois</td></tr>
<tr ><td><span >?</span></td><td>Rendre le motif facultatif en l'autorisant à correspondre 0 ou 1 fois.</td></tr>
<tr ><td><span >\+</span></td><td>Exiger que le motif corresponde une ou plusieurs fois</td></tr>
<tr ><td><span >\*</span></td><td>Permettre au modèle de correspondre à zéro ou plusieurs fois</td></tr>
</table>


### Attention aux lemmes !
Si nous voulions trouver une correspondance à la fois pour "solar power" et "solar powered", il serait tentant de chercher le *lemme* de "powered" et de s'attendre à ce qu'il s'agisse de "power". Ce n'est pas toujours le cas ! Le lemme de l'*adjectif* "powered" est toujours "powered" :

In [26]:
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP':'*'}, {'LEMMA': 'power'}] # CHANGE THIS PATTERN

# Remove the old patterns to avoid duplication:
matcher.remove('SolarPower')

# Add the new set of patterns to the 'SolarPower' matcher:
matcher.add('SolarPower', [pattern1, pattern2])

In [27]:
doc2 = nlp(u'Solar-powered energy runs solar-powered cars.')

In [28]:
found_matches = matcher(doc2)
print(found_matches)

[(8656102463236116519, 0, 3), (8656102463236116519, 5, 8)]


<font color=green>Le matcher a trouvé la première occurrence parce que le lemmatiseur a traité 'Solar-powered' comme un verbe, mais pas la seconde car il l'a considéré comme un adjectif.<br>Pour ce cas, il peut être préférable de définir des modèles de tokens explicites.</font>

In [29]:
pattern1 = [{'LOWER': 'solarpower'}]
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP':'*'}, {'LOWER': 'power'}]
pattern3 = [{'LOWER': 'solarpowered'}]
pattern4 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP':'*'}, {'LOWER': 'powered'}]

# Remove the old patterns to avoid duplication:
matcher.remove('SolarPower')

# Add the new set of patterns to the 'SolarPower' matcher:
matcher.add('SolarPower', [pattern1, pattern2, pattern3, pattern4])

In [30]:
found_matches = matcher(doc2)
print(found_matches)

[(8656102463236116519, 0, 3), (8656102463236116519, 5, 8)]


## Autres attributs du token
Outre les lemmes, il existe une variété d'attributs de jetons que nous pouvons utiliser pour déterminer les règles de correspondance :
<table><tr><th>Attribute</th><th>Description</th></tr>

<tr ><td><span >`ORTH`</span></td><td>The exact verbatim text of a token</td></tr>
<tr ><td><span >`LOWER`</span></td><td>The lowercase form of the token text</td></tr>
<tr ><td><span >`LENGTH`</span></td><td>The length of the token text</td></tr>
<tr ><td><span >`IS_ALPHA`, `IS_ASCII`, `IS_DIGIT`</span></td><td>Token text consists of alphanumeric characters, ASCII characters, digits</td></tr>
<tr ><td><span >`IS_LOWER`, `IS_UPPER`, `IS_TITLE`</span></td><td>Token text is in lowercase, uppercase, titlecase</td></tr>
<tr ><td><span >`IS_PUNCT`, `IS_SPACE`, `IS_STOP`</span></td><td>Token is punctuation, whitespace, stop word</td></tr>
<tr ><td><span >`LIKE_NUM`, `LIKE_URL`, `LIKE_EMAIL`</span></td><td>Token text resembles a number, URL, email</td></tr>
<tr ><td><span >`POS`, `TAG`, `DEP`, `LEMMA`, `SHAPE`</span></td><td>The token's simple and extended part-of-speech tag, dependency label, lemma, shape</td></tr>
<tr ><td><span >`ENT_TYPE`</span></td><td>The token's entity label</td></tr>

</table>

### Caractère de remplacement du token
Vous pouvez passer un dictionnaire vide `{}` comme joker pour représenter **n'importe quel token**. Par exemple, vous pourriez vouloir récupérer des hashtags sans savoir ce qui pourrait suivre le caractère `#` :
>`[{'ORTH': '#'}, {}]`

## PhraseMatcher
Dans la section précédente, nous avons utilisé des modèles de tokens pour effectuer des correspondances basées sur des règles. Une méthode alternative - et souvent plus efficace - consiste à faire correspondre des listes terminologiques. Dans ce cas, nous utilisons PhraseMatcher pour créer un objet Doc à partir d'une liste de phrases, et le passer dans `matcher` à la place.

In [31]:
# Perform standard imports, reset nlp
import spacy
nlp = spacy.load('en_core_web_sm')

In [32]:
# Import the PhraseMatcher library
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)

Pour cet exercice, nous allons importer un article de Wikipedia sur *Reaganomics*<br>
Source: https://en.wikipedia.org/wiki/Reaganomics

In [33]:
# with open('../TextFiles/reaganomics.txt', encoding='utf8') as f:
#     doc3 = nlp(f.read())

with open('../data/TextFiles/reaganomics.txt', encoding='ISO-8859-1') as f:
    doc3 = nlp(f.read())

In [35]:
# First, create a list of match phrases:
phrase_list = ['voodoo economics', 'supply-side economics', 'trickle-down economics', 'free-market economics']

# Next, convert each phrase to a Doc object:
phrase_patterns = [nlp(text) for text in phrase_list]

# Pass each Doc object into matcher (note the use of the asterisk!):
matcher.add('VoodooEconomics', [*phrase_patterns])

# Build a list of matches:
matches = matcher(doc3)

In [37]:
# (match_id, start, end)
matches

[(3473369816841043438, 41, 45),
 (3473369816841043438, 49, 53),
 (3473369816841043438, 54, 56),
 (3473369816841043438, 61, 65),
 (3473369816841043438, 673, 677),
 (3473369816841043438, 2986, 2990)]

<font color=green>Les quatre premières correspondances sont celles où ces termes sont utilisés dans la définition des "Reaganomics" :</font>

In [38]:
doc3[:70]

REAGANOMICS
https://en.wikipedia.org/wiki/Reaganomics

Reaganomics (a portmanteau of [Ronald] Reagan and economics attributed to Paul Harvey)[1] refers to the economic policies promoted by U.S. President Ronald Reagan during the 1980s. These policies are commonly associated with supply-side economics, referred to as trickle-down economics or voodoo economics by political opponents, and free-market economics by political advocates.


## Visualisation des correspondances
Il y a plusieurs façons de récupérer le texte entourant une correspondance. La plus simple est de récupérer une tranche de mots de la documentation qui est plus large que la correspondance :

In [39]:
doc3[665:685]  # Note that the fifth match starts at doc3[673]

same time he attracted a following from the supply-side economics movement, which formed in opposition to Keynesian

In [40]:
doc3[2975:2995]  # The sixth match starts at doc3[2985]

lawsuits against institutions.[66] His policies became widely known as "trickle-down economics", due to the

Une autre méthode consiste à appliquer d'abord le `sentencizer` au Doc, puis à parcourir les phrases jusqu'au point de correspondance :

In [41]:
# Build a list of sentences
sents = [sent for sent in doc3.sents]

# In the next section we'll see that sentences contain start and end token values:
print(sents[0].start, sents[0].end)

0 35


In [42]:
# Iterate over the sentence list until the sentence end value exceeds a match start value:
for sent in sents:
    if matches[4][1] < sent.end:  # this is the fifth match, that starts at doc3[673]
        print(sent)
        break

At the same time he attracted a following from the supply-side economics movement, which formed in opposition to Keynesian demand-stimulus economics.


Pour plus d'informations, consultez le site https://spacy.io/usage/linguistic-features#section-rule-based-matching
## Suivant : Évaluation